# Convert japanese DEMs to geotiff
This notebook converts Japanese DEM XML files into Geotiffs using the CLI tool "jpgis-dem" in Python.  
Installation instructions are in the links below:  
- Blog: https://www.gpxz.io/blog/jpgis-dem  
- Github repository: https://github.com/gpxz/jpgis-dem/tree/main?tab=readme-ov-file  

### Environment

In [ ]:
import os
import zipfile
import shutil
import numpy as np
import pytest
import rasterio

import jpgisdem
from fix_dghm_tags import *

#print("Working repository :", os.getcwd())

***/!\\ rasterio version***     
There might be issues with the rasterio version between the one used by jpgis-dem and the one from PostGIS/PostgreSQL.  
If there are issue, make sure to check which rasterio we are working with. The version to be used can be specified with:   
`setx PROJ_LIB "[virtualenv_path]\rasterio\proj_data"`

In [ ]:
# example command to visu which rasterio is used:
!python -c "import rasterio; print(rasterio._env.get_proj_data_search_paths())"

## Running the tests (optional)
Running the function on the test files provided on the github.

In [ ]:
# checking filepath
file_path = r".\tests\test_jpgisdem.py"
if os.path.exists(file_path):
    print(f"Le fichier existe : {file_path}")
else:
    print(f"Le fichier n'existe pas : {file_path}")

In [ ]:
!pytest -v .\tests\test_jpgisdem.py

In [ ]:
# converting the tests files
!jpgis-dem xml2tif .\tests\data\FG-GML-3624-30-DEM5B.zip .\tests\export\FG-GML-3624-30-DEM5B.geotiff

Note: might need to update the test_jpgisdem.py and specify "rb" in `with open():`.  
e.g., `with open(XML_path, "rb") as src, ...`

## Convert a single file or single zip folder
For zips, return a single geotiff which merges all files within the zip.

In [ ]:
# replace dghm with dems
# specify --zip for zip files
!python .\fix_dghm_tags.py .\data\all_DEMs\Tohoku\GMLdata_akita.zip .\data\all_DEMs\Tohoku\patched_tohoku_akita.zip --zip    

In [ ]:
# convert to geotiff
!jpgis-dem xml2tif .\data\all_DEMs\Tohoku\patched_tohoku_akita.zip .\data\all_DEMs\Tohoku\tohoku_akita.geotiff

## Convert all XMLs within a folder

In [ ]:
folder_path = r".\data\all_DEMs\Hokkaido\GMLdata_00hokkaido"
# checking filepath
if os.path.exists(folder_path):
    print(f"Folder exists : {folder_path}")
else:
    print(f"Folder does not exist : {folder_path}")

In [ ]:
# loop through XMLs
for root, dirs, files in os.walk(folder_path):
    for file in files:
        # output folders
        file_path = os.path.join(root,file)
        print(file_path)
        patched_dir = os.path.join(root, "patched")
        geotiff_dir = os.path.join(root, "geotiff")
        os.makedirs(patched_dir, exist_ok=True)
        os.makedirs(geotiff_dir, exist_ok=True)
        
        #output paths
        print(patched_dir)
        patched_path = os.path.join(patched_dir, f"{file[:-4]}_patched.xml")
        geotiff_path = os.path.join(geotiff_dir, f"{file[:-4]}.geotiff")

        if os.path.exists(patched_path):
            if os.path.isdir(patched_path):
                shutil.rmtree(patched_path)
            else:
                os.remove(patched_path)

        if os.path.exists(geotiff_path):
            if os.path.isdir(geotiff_path):
                shutil.rmtree(geotiff_path)
            else:
                os.remove(geotiff_path)

        # patch
        !python .\fix_dghm_tags.py "{file_path}" "{patched_path}"

        # convert to geotiff
        !jpgis-dem xml2tif "{patched_path}" "{geotiff_path}"

        # remove empty folders
        for d in [patched_dir, geotiff_dir]:
            if os.path.isdir(d) and not os.listdir(d):
                os.rmdir(d)

## Convert all ZIPs within a folder

In [ ]:
folder_path = r".\data\all_DEMs"
# checking filepath
if os.path.exists(folder_path):
    print(f"Folder exists : {folder_path}")
else:
    print(f"Folder does not exist : {folder_path}")

In [ ]:
# loop through zips
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith(".zip"):
            zip_path = os.path.join(root,file)
            print(zip_path)
            
            # output folders
            patched_dir = os.path.join(root, "patched")
            geotiff_dir = os.path.join(root, "geotiff")
            os.makedirs(patched_dir, exist_ok=True)
            os.makedirs(geotiff_dir, exist_ok=True)

            #output paths
            patched_path = os.path.join(patched_dir, f"{file[:-4]}_patched.zip")
            geotiff_path = os.path.join(geotiff_dir, f"{file[:-4]}.geotiff")

            if os.path.exists(patched_path):
                if os.path.isdir(patched_path):
                    shutil.rmtree(patched_path)
                else:
                    os.remove(patched_path)

            if os.path.exists(geotiff_path):
                if os.path.isdir(geotiff_path):
                    shutil.rmtree(geotiff_path)
                else:
                    os.remove(geotiff_path)

            # patch
            !python .\fix_dghm_tags.py "{zip_path}" "{patched_path}" --zip

            # convert to geotiff
            !jpgis-dem xml2tif "{patched_path}" "{geotiff_path}"

            # remove empty folders
            for d in [patched_dir, geotiff_dir]:
                if os.path.isdir(d) and not os.listdir(d):
                    os.rmdir(d)
            
        
    